In [3]:
import datetime # Imports module for working with date&time
import os # Imports the module for working with reading and writing to the file
import shutil # Imports from the module for working with collecting file

#Global Variables 
STUDENT_ID = "850003235" #Adding student number
MAIN_FILE = f"LHMS_{STUDENT_ID}.txt" # store the allocated rooms in the dictionary 

# In-memory data structures for rooms and allocations
rooms = [] 
allocated_rooms = {} 
# Dictionary for room types
room_types = { #Showing room type, price, and different features 
    "Standard": {"price": 700, "features": ["TV", "Fan"]},     # $700 per week ($100/day)
    "Deluxe": {"price": 1400, "features": ["TV", "Aircon", "WI-FI"]}, # $1400 per week ($200/day)
    "Suite": {"price": 2450, "features": ["TV", "Aircon", "WI-FI", "Balcony"]} # $2450 per week ($350/day)
}
#  Helper Functions
def reset_globals(): #Function to reset global variables
#
    global rooms, allocated_rooms #shows that we will be modifying these
    rooms = [] #empty lisit for storing rooms
    allocated_rooms = {} #empty dictionary
    try: #try block to prevent errors
        if os.path.exists(MAIN_FILE):#checking if the file exsits
            os.remove(MAIN_FILE)#atemmp to delete this file 
            print(f"File '{MAIN_FILE}' removed successfully.") #file removed successfully 
    except OSError as e: #catches OS errors if it fails
        print(f"Error removing file '{MAIN_FILE}': {e}")#messsage if file is removed
    print("\n Global state reset")#comfiring the reset


In [4]:
### MENU OPTIONS
def display_menu(): 
    "Prints the main menu options for the Hotel Management System."
    print("\nLANGHAM HOTEL MANAGEMENT SYSTEM MENU") #header for the menu
    print(" 1 - Add Rooms")#option to add rooms
    print(" 2 - Delete Rooms")#option to delete roooms
    print(" 3 - Display Room Details")#option to display room details
    print(" 4 - Allocate Rooms")#option to allocate a room
    print(" 5 - Display Room Allocation Details")#option to display room allocation
    print(" 6 - Billing & De-Allocation")#option to de-allocate and bill guest
    print(" 7 - Save Room Allocations To a File")#option to save room allocation to the file
    print(" 8 - Show Room Allocations From a File")#option to show room allocation from the file
    print(" 9 - Backup and Clear Room Allocation File")#option to backup and clear original file
    print(" 0 - Exit")#exit the system

In [5]:
def get_validated_int_input(prompt, min_value=1): 
#forces the user to use an integer with a minimum value of 1
    while True: #loop until the input is recived
        user_input = input(prompt).strip() 
        
        try:
            value = int(user_input) # convert input to an integer
        except ValueError: #handles non-integer
            print("Invalid input. Please enter a whole number.") 
            continue # restart the loop
        if value < min_value: #checking if the integer meets the minimum value
            print(f"The number must be at least {min_value}.")
            continue #restart the loop
        return value #returns the right integer
        
def add_room(room_number): #Defines the function that adds a new room and number
    print("\nAdd Room") #add room as a heading
    print("Available Room Types:")#lets the user know of the next output
    for rt, details in room_types.items():
        #displays each room type, features, and weekly price
        print(f"- {rt} (Weekly Price: ${details['price']}, Features: {', '.join(details['features'])})")

    room_type = input(f"Enter room type for {room_number}: ").strip()#user to add room type

    try: #try block to stop an error 
        if room_number in rooms: #checks if the room number exsits in the list
            print(f"Room {room_number} already exists.") #shows that the room already exsits
            return #exit function if it exists

        if room_type not in room_types: #checks if room type is corret
            #raises a error if user picks wrong room type
            raise ValueError("Invalid room type. Please choose from Standard, Deluxe, or Suite.")

        rooms.append(room_number) #if both room type and room number are correct, add it 
        allocated_rooms[room_number] = { # adding room number to allocated room
            "type": room_type, #stores room type
            "price": room_types[room_type]["price"],#stores weekly price
            "features": room_types[room_type]["features"],#stores room fetures from room type
            "guest": None, #showing the room is unoccupied
            "days_staying": 0 #days staying is 0
        }
        print(f"Room {room_number} ({room_type}) added successfully.") #if added corret it adds it successfully
    except ValueError as ve: #catches a ValueError
        print(f"Error adding room: {ve}") #displays an error message
    except Exception as e: #catches any other erros
        print(f"An unexpected error occurred while adding room: {e}") #prints a general message for other erros


In [6]:
def delete_room(room_number): #Defines the function that deletes a room and number

    print(f"\n Delete Room {room_number}") #displays which room is being deleted
    try: #try block where errors will be monitored
        if room_number not in rooms: #checks if the room exsits
            raise KeyError(f"Room {room_number} does not exist in the system.")#Key error if room is not there
        
        if room_number in allocated_rooms and allocated_rooms[room_number].get("guest"):
            #checks if the room is allocated
            print(f"Error: Room {room_number} is currently occupied by {allocated_rooms[room_number]['guest']}. Cannot delete.")
            # can't delete an occupied room
            return #exits a function if the room is occupied

        rooms.remove(room_number) #if the room is removed, it checks for the room number
        if room_number in allocated_rooms: #if the room is in allocated rooms
            del allocated_rooms[room_number] #deletes details from allocated room
        print(f"Room {room_number} deleted successfully.") #room deleted successfully
    except KeyError as ke: #catches a key error
        print(f"Error deleting room: {ke}")#if the room doesnt exsits 
    except ValueError as ve:#catches a ValueError
        print(f"Error deleting room: {ve}")#displays an error message
    except Exception as e:#catches any other errors
        print(f"An unexpected error occurred while deleting room: {e}")#prints a general message for other errors

In [7]:

def display_rooms(): # Function to display details of all rooms
    print("\nDisplay Room Details") # Print a header
    if not rooms: # Check if the rooms list is empty
        print("No rooms available.") # tells user if no rooms
        return # Exit function

    print("\nRoom Details:") # Print room details
    for room in rooms: # Loop each room number in the list
        room_info = allocated_rooms.get(room) # Get detailed details from room
        if room_info: # continues if room information was found
            try: # try block for error
                room_type = room_info.get('type', 'N/A') # gets room type
                room_price_weekly = room_info.get('price', 'N/A') # gets weekly price
                
                features_list = room_info.get('features', []) # Get list of features
               
                if not isinstance(features_list, list): # checks if feature list is a list
                    features_str = "Corrupted Feature Data" # default string if data is corrupted
                else:
                    features_str = ", ".join(map(str, features_list)) # Join features into a string
                
                guest_status = "Unoccupied" # displays status as unoccupied
                if room_info.get('guest'): # Check if guest exists and has a value
                    # status is set with name and days staying
                    guest_status = f"Occupied by {room_info['guest']} for {room_info.get('days_staying', 'N/A')} days"

                # Print all room details 
                print(f"Room {room}: {room_type} | Weekly Price: ${room_price_weekly} | Features: {features_str} | Status: {guest_status}")
            
            except KeyError as ke: # Catch KeyError
                print(f"Warning: Missing data for room {room} (KeyError: {ke}). Displaying partial info.")
            except TypeError as te: # Catch TypeError
                print(f"Warning: Corrupted data type for room {room} (TypeError: {te}). Displaying partial info.")
            except Exception as e: # Catch any other errors
                print(f"An unexpected error when displaying room {room}: {e}")
        else:
            # If room_info was None
            print(f"Room {room}: Details not fully set up (no allocated_rooms entry).")

In [8]:
# Defines a function to allocate a room
def allocate_room(room_number, guest_name):

    print(f"\nAllocate Room {room_number} to {guest_name}")
    # Print a header with room and guest
    try: #try block for error handling
        if room_number not in rooms: # Check if the room exists
            raise KeyError(f"Room {room_number} does not exist. Please add it.") #KeyError if room does not exist
        room_info = allocated_rooms.get(room_number)
        if not room_info: #details from the room number
            raise ValueError(f"Internal Error: Details for room {room_number} are incomplete. Please re-add the room.")
        if room_info.get("guest"): # Check if the room is already occupied 
            # let user know if the room is already occupied
            print(f"Room {room_number} is already occupied by {room_info['guest']}.")
            return
        current_room_type = room_info.get('type', 'Unknown Type')# show the room type for display
        print(f"Room {room_number} is a {current_room_type} room.")# Display room type
        # Get integer input for days staying
        days_staying = get_validated_int_input("Enter number of days staying: ", min_value=1)
 
        room_info["guest"] = guest_name #get guest name
        room_info["days_staying"] = days_staying #get how many days the guest is staying
# confirming message
        print(f"Room {room_number} ({current_room_type}) allocated to {guest_name} successfully for {days_staying} days.")

    except KeyError as ke: # Catch KeyError
        print(f"Error allocating room: {ke}") # Print KeyError message
    except ValueError as ve: # Catch ValueError
        print(f"Error allocating room: {ve}") # show ValueError message
    except Exception as e: #unexpected errors
        print(f"An unexpected error occurred while allocating room: {e}")


In [9]:
 # Defines the display details of allocated rooms function

def display_allocated_rooms():

    print("\nDisplay Allocated Rooms") # Prints a header
    found_allocated = False #track if any allocated rooms are found
    
    for room_num, details in allocated_rooms.items():#Loop through details in the allocatedrooms dictionary
        if details.get("guest"): # Check if the guest exists
            found_allocated = True #True if allocated room is found
            try:
                room_type = details.get('type', 'Unknown')  #gets the room type default Unknown
                guest_name = details.get('guest', 'N/A')# Get guest name default N/A
                room_price_weekly = details.get('price', 'N/A')# Get weekly price
                days_staying = details.get('days_staying', 'N/A')# Get days staying
                
                features_list = details.get('features', [])# Get features list
                if not isinstance(features_list, list): # sees if features list is not a list
                    features_str = "Corrupted Feature Data"
                    print(f"Warning: Corrupted features data for room {room_num}.")
                else:
                    features_str = ", ".join(map(str, features_list))  # Join features into string 
                  # display details for the allocated room  
                print(f"Room {room_num} ({room_type}) - Guest: {guest_name} - "
                      f"Weekly Price: ${room_price_weekly} - Days Staying: {days_staying} - "
                      f"Features: {features_str}")
            except KeyError as ke:  # Catch KeyError
                print(f"Warning: Missing data for allocated room {room_num} (KeyError: {ke}). Displaying partial info.")
            except TypeError as te:  # Catch TypeError
                print(f"Warning: Corrupted data type for allocated room {room_num} (TypeError: {te}). Displaying partial info.")
            except Exception as e: # Catch any other error
                print(f"An unexpected error occurred displaying allocated room {room_num}: {e}")
    
    if not found_allocated: 
        print("No rooms currently allocated.")               


In [10]:
# Defines a function to deallocate a room and billing
def deallocate_room(room_number):

    print(f"\nDeallocate Room {room_number}") #header room and number

    try:
        if room_number not in allocated_rooms: # Check if the room number exists
             # If not found raise a KeyError
            raise KeyError(f"Room {room_number} does not exist in allocations.")

        room_info = allocated_rooms[room_number]# Get the information from dictionary 
        if not room_info.get("guest"):# Checks if the room is currently occupied
            raise ValueError(f"Room {room_number} is not currently occupied.")
        guest_name = room_info['guest']# Get the name of the guest
        room_type = room_info.get('type', 'N/A') # Get the room type
        
        features_list = room_info.get('features', [])# Get the list of features
        if not isinstance(features_list, list): # Check if the features are not a list
            features_str = "Corrupted Feature Data"
        else:
            features_str = ", ".join(map(str, features_list))# Join features into a string
        
        weekly_price = room_info.get('price', 0)# Get the rooms weekly price
        days_stayed = room_info.get('days_staying', 0)# Get the number of days

        total_charge = 0 #total charge for billing
        billing_denominator = 7 #number of days in a week for daily rate calculation
        try:
             # Check if price or days stayed are zero
            if weekly_price == 0 or days_stayed == 0:
                print("Warning: Price or days stayed information is missing/zero. Cannot calculate bill.")
                total_charge = 0
            else: #if not 0 calculate daily price
                daily_price = weekly_price / billing_denominator #weeky price divied by days stayed
                total_charge = round(daily_price * days_stayed, 2)
        except ZeroDivisionError:# Catch ZeroDivisionError
            print(f"Error: Cannot calculate bill for room {room_number} due to division by zero (e.g., trying to divide by zero days in a week).")
            total_charge = 0 
        room_info["guest"] = None #this makes room unoccupied 
        room_info["days_staying"] = 0 #resets days to 0
        #room is not unoccupied
        print(f"Room {room_number} released. Was occupied by {guest_name}.")#show who it was occupied by
        print(f"Billing Details: Room Type {room_type}, Features: {features_str}.")# Print billing details
        print(f"Weekly Rate: ${weekly_price}, Days Stayed: {days_stayed}.")
        print(f"Total Charge: ${total_charge}.") #show total charge
    except KeyError as ke:#catch KeyError
        print(f"Error deallocating room: {ke}")
    except ValueError as ve:# Catch ValueError
        print(f"Error deallocating room: {ve}")
    except Exception as e:# Catch other errors 
        print(f"An unexpected error occurred while deallocating room: {e}")


In [11]:
#a function to save allocated room details to a file
def save_allocations_to_file():
    print("\nSave Allocations to File")
    # the number of currently allocated rooms 
    allocated_count = sum(1 for details in allocated_rooms.values() if details.get("guest"))
    if allocated_count == 0:# Check if there are no rooms allocated
        print("No rooms currently allocated to save.")#displays no room allocated
        return # Exit function if theres nothing to save

    try:
        with open(MAIN_FILE, "w") as file:# Opens file in write mode 
            for room, details in allocated_rooms.items():#goes through room deatils
                guest_name = details.get("guest") # Get the guest's name for the current room
                if guest_name:#continue only if the room is currently occupied
                    room_type = details.get('type', 'Unknown')
                    room_price_weekly = details.get('price', 'N/A')
                    days_staying = details.get('days_staying', 'N/A')
                    
                    features_list = details.get('features', [])
                    if not isinstance(features_list, list):
                        features_str = "Corrupted Feature Data"
                        print(f"Warning: Corrupted features data for room {room} during save.")
                    else:
                         # Convert all features to string
                        features_str = ", ".join(map(str, features_list)) 
                    file.write(f"Room {room}, Type: {room_type}, Guest: {guest_name}, Weekly Price: ${room_price_weekly}, Days Staying: {days_staying}, Features: {features_str}\n")
        print(f"Room allocations saved successfully to '{MAIN_FILE}'.")
    except IOError as e: #Catch a IOE Error
        print(f"Error saving allocations to file '{MAIN_FILE}': {e}. Check permissions or disk space.")
    except Exception as e:#Catchs any other errors
        print(f"An unexpected error occurred during saving allocations: {e}")


In [14]:
#a function to display room allocations read from a file
def display_allocations_from_file():

    print("\nDisplay Allocations From File")
    try:
        with open(MAIN_FILE, "r") as file:
            content = file.read() # Reads content of the file
            if not content.strip():# Checks if there is conent or a black space
                print(f"The file '{MAIN_FILE}' is empty.") #let user know if its empty
            else:
                print("\nRoom Allocations from File:\n" + content)#show content details
    except FileNotFoundError: #catches a file not found error
        print(f"Error: The file '{MAIN_FILE}' was not found. No records available.")
    except IOError as e: # Catch any other IOError
        print(f"An I/O error occurred with file '{MAIN_FILE}': {e}. Could not display contents.")
    except Exception as e:
        print(f"An unexpected error occurred during display from file: {e}")


In [15]:
def backup_and_clear_allocations(): #a function to back up and clear the allocation file

    print("\nBackup and Clear Allocations")
     # Creates a backup filename with student number and timestamp
    backup_filename = f"LHMS_{STUDENT_ID}_Backup_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt"
    
    try: #a try block for file operations
        if not os.path.exists(MAIN_FILE): ## Check if the original file exists
            print(f"Error: Original file '{MAIN_FILE}' does not exist to back up. No backup performed.")
            return
        
        shutil.copy(MAIN_FILE, backup_filename) # Copy the file to the newly created backup filename
        print(f"Backup saved as '{backup_filename}'.")

        open(MAIN_FILE, "w").close() # Open the main file in write mode
        print(f"Original allocation file '{MAIN_FILE}' has been cleared.")
    except FileNotFoundError as e: #catch file not found
        print(f"Error backing up: {e}. The file might have been removed just before copying.")
    except IOError as e: # Catch any other IOError
        print(f"An I/O error occurred during backup or clear operation: {e}")
    except Exception as e:#catches any other error
        print(f"An unexpected error occurred during backup and clear: {e}")


In [ ]:
#MAIN PROGRAM BEGINS
def main(): # Defines the main function
 
    print("Welcome to Langham Hotel Management System!")#header
    while True: # starts loop to keep the program running
        display_menu() # Calls the function display menu
        try:
            # asks the user to enter their choice number
            choice = input("Enter Your Choice Number: ").strip()
        except EOFError: # Catches EOFError
            print("\nEnd of input detected. Exiting system.")
            break # Exits the infinite loop
        except KeyboardInterrupt: # Catches KeyboardInterrupt
            print("\nProgram interrupted by user. Exiting system.")
            break# Exits the infinite loop
        except Exception as e: #catches other errors
            print(f"An unexpected error occurred : {e}. Please try again.")
            continue # Continues to the next iteration of the while loop

        if choice == "1": # If option 1 is chosen, add a room
            room_number = input("Enter Room Number to Add: ") # Add the room number
            add_room(room_number)# Calls add room to add it
        elif choice == "2":# If option 2 is chosen, delete the room
            room_number = input("Enter Room Number to Delete: ")# Enter the room number
            delete_room(room_number)# Deletes the room number
        elif choice == "3":# If option 3 is chosen
            display_rooms()# Display rooms shown
        elif choice == "4": # If option 4 is chosen to allocate rooms
            room_number = input("Enter Room Number to Allocate: ") # Add the room number you want to allocate
            guest_name = input("Enter Guest Name: ")# add guest name
            allocate_room(room_number, guest_name)# Calls the function to allocate room to the guest
        elif choice == "5":# If option 5 is chosen to display allocated rooms
            display_allocated_rooms() # Calls the function to display allocated rooms
        elif choice == "6":# If option 6 is chosen to De-allocate and billing 
            room_number = input("Enter Room Number to De-allocate: ")# add room number for deallocation
            deallocate_room(room_number)# Calls the function to de-allocate 
        elif choice == "7": #If option 7 is chosen to save allocation to file
            save_allocations_to_file()# Calls the function to save the current allocations
        elif choice == "8":#If option 8 is chosen
            display_allocations_from_file()# Calls the function to display allocations from file
        elif choice == "9":#If option 9 is chosen
            backup_and_clear_allocations()# Calls the function to back up and clear 
        elif choice == "0":#If option 0 is chosen. Exit system
            print("Exiting Hotel Management System. Thank you .Goodbye!")
            break # Exits the infinite while loop
        else: # If the user's choice does not match any of the options
            print("Invalid choice. Please enter a number from the menu.")

# Run main program
if __name__ == "__main__":
    main() # Calls the main function to start

Welcome to Langham Hotel Management System!

LANGHAM HOTEL MANAGEMENT SYSTEM MENU
 1 - Add Rooms
 2 - Delete Rooms
 3 - Display Room Details
 4 - Allocate Rooms
 5 - Display Room Allocation Details
 6 - Billing & De-Allocation
 7 - Save Room Allocations To a File
 8 - Show Room Allocations From a File
 9 - Backup and Clear Room Allocation File
 0 - Exit


Enter Your Choice Number:  1
Enter Room Number to Add:  1



Add Room
Available Room Types:
- Standard (Weekly Price: $700, Features: TV, Fan)
- Deluxe (Weekly Price: $1400, Features: TV, Aircon, WI-FI)
- Suite (Weekly Price: $2450, Features: TV, Aircon, WI-FI, Balcony)


Enter room type for 1:  Standard


Room 1 (Standard) added successfully.

LANGHAM HOTEL MANAGEMENT SYSTEM MENU
 1 - Add Rooms
 2 - Delete Rooms
 3 - Display Room Details
 4 - Allocate Rooms
 5 - Display Room Allocation Details
 6 - Billing & De-Allocation
 7 - Save Room Allocations To a File
 8 - Show Room Allocations From a File
 9 - Backup and Clear Room Allocation File
 0 - Exit


...

Invalid choice. Please enter a number from the menu.

LANGHAM HOTEL MANAGEMENT SYSTEM MENU
 1 - Add Rooms
 2 - Delete Rooms
 3 - Display Room Details
 4 - Allocate Rooms
 5 - Display Room Allocation Details
 6 - Billing & De-Allocation
 7 - Save Room Allocations To a File
 8 - Show Room Allocations From a File
 9 - Backup and Clear Room Allocation File
 0 - Exit


Enter Your Choice Number:  


Invalid choice. Please enter a number from the menu.

LANGHAM HOTEL MANAGEMENT SYSTEM MENU
 1 - Add Rooms
 2 - Delete Rooms
 3 - Display Room Details
 4 - Allocate Rooms
 5 - Display Room Allocation Details
 6 - Billing & De-Allocation
 7 - Save Room Allocations To a File
 8 - Show Room Allocations From a File
 9 - Backup and Clear Room Allocation File
 0 - Exit


Enter Your Choice Number:  Z


Invalid choice. Please enter a number from the menu.

LANGHAM HOTEL MANAGEMENT SYSTEM MENU
 1 - Add Rooms
 2 - Delete Rooms
 3 - Display Room Details
 4 - Allocate Rooms
 5 - Display Room Allocation Details
 6 - Billing & De-Allocation
 7 - Save Room Allocations To a File
 8 - Show Room Allocations From a File
 9 - Backup and Clear Room Allocation File
 0 - Exit
